In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as d

In [2]:
#Parameters
samplenum = 2
input_size = 3
output_size = 3
time_length = 3; #ms

In [3]:
# Generate simulation
dyn = d.PyDyn('../Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)
#sample p
p = np.ones((3*time_length, samplenum))
y_target = np.zeros((samplenum,3))
y_target[:,2] = np.random.rand(samplenum)
y_target[:,1] = 2
y_target = np.array([0, 2, 0.5])
#for i in range(samplenum):
    #p[:,i] = dyn.get_p(y_target.transpose()[:,i], dyn.p_init)
p[:,1] = dyn.p_init
p[:,0] =p[:,1]
print(p)

[[0. 0.]
 [3. 3.]
 [0. 0.]
 [0. 0.]
 [3. 3.]
 [0. 0.]
 [0. 0.]
 [3. 3.]
 [0. 0.]]


## Building the custon Simulation Activation Function

In [4]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        y_pred = torch.ones([samplenum,3])
        for i in range(len(p[0, :])):
            state = dyn.compute(p[:,i])
            y_pred[i, :] = torch.tensor(state.y[-3:])
        #print(f'y_pred: {y_pred.shape}')
        
        ctx.save_for_backward(input)
        
        return y_pred
        
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        dy_dp_batch = torch.zeros([3, 3*time_length])
        for i in range(samplenum):
            state= dyn.compute(p[:, i])
            dy_dp = dyn.dy_dp(state, p[:, i])
            dy_dp = torch.tensor(dy_dp[-3:, :])
            dy_dp_batch = dy_dp_batch + dy_dp
        #print(f'shape of dy/dp_batch: {dy_dp_batch.shape}')
        
        grad_input = torch.tensor(dy_dp_batch).t().mm(grad_output.t().double()).t()
        #print(f'shape of grad input: {grad_input.shape}')
        #print(f'shape of grad output: {grad_output.shape}')
        print(grad_input)
        return grad_input    

Simulate = Simulate.apply



In [5]:
from torch.autograd import gradcheck

# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
#p = dyn.p_init
p = torch.tensor(p, requires_grad = True).t()
input = (p.double())
#print(input)
test = gradcheck(Simulate, (input,), eps=1e-3, atol=1e-3, raise_exception = True)
print(test)

tensor([[ 1.6197e-02,  2.9054e-06,  0.0000e+00,  1.0838e-02, -7.5657e-06,
          0.0000e+00,  5.4336e-03, -3.8690e-05,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
       dtype=torch.float64)
tensor([[ 1.6197e-02,  2.9054e-06,  0.0000e+00,  1.0838e-02, -7.5657e-06,
          0.0000e+00,  5.4336e-03, -3.8690e-05,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
       dtype=torch.float64)
tensor([[ 2.6722e-06, -1.0790e-01,  0.0000e+00, -7.5027e-06,  3.5867e-01,
          0.0000e+00, -3.8690e-05,  1.8008e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
       dtype=torch.float64)
tensor([[ 2.6722e-06, -1.0790e-01,  0.0000e+00, -7.5027e-06,  3.5867e-01,
          0.0000e+00, -

RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[ 8.1001e-03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 1.4525e-06, -5.3942e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  8.1001e-03,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 5.4197e-03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-3.7826e-06,  1.7929e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  5.4197e-03,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 2.7172e-03, -5.9605e-05,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [-1.9346e-05,  9.0039e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  2.7172e-03,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,

In [15]:
print(test.backward(torch.ones(1,3)))

AttributeError: 'bool' object has no attribute 'backward'